In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
issue = pd.read_csv("../data/fb_22_output_multi_25.csv.gz")

In [3]:
issue.head(2)

,text,id,ISSUE215,ISSUE10,ISSUE30,ISSUE40,ISSUE212,ISSUE12,ISSUE16,ISSUE209,...,ISSUE91,ISSUE56,ISSUE53,ISSUE90,ISSUE65,ISSUE45,ISSUE60,ISSUE58,ISSUE22,ISSUE32
0,',x_4971277182972491__ad_creative_link_descripti...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,' YOUR SOURCE FOR @GANDERNEWSROOM And I very m...,x_605605801303782__aws_ocr_text_vid,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Function to get column names where value is 1 for the last 25 columns
def get_active_features(row):
    active_features = issue.columns[-25:][row.iloc[-25:] == 1]
    return ','.join(active_features)

In [5]:
# Apply the function to each row
issue['issue'] = issue.apply(get_active_features, axis=1)

In [6]:
# Split ids
issue['id'] = issue['id'].str.split('|')
# "Un-deduplicate", or "Re-hydrate", in WMP lingo
issue = issue.explode('id')
# Split into ad id and field
issue_ids = issue['id'].str.split('__', expand = True)
issue_ids.columns = ['ad_id', 'field']
issue = pd.concat([issue, issue_ids], axis = 1)
issue = issue.drop(labels = ['id'], axis = 1)
issue = issue[['ad_id', 'field', 'issue']]

In [7]:
issue.rename(columns={'field': 'issue_field', 'issue': 'issue_class'}, inplace=True)

In [8]:
# Replace various representations of missing values with NaN
missing_values = ['', None, 'NA', 'missing']
issue['issue_class'] = issue['issue_class'].replace(missing_values, np.nan)
issue = issue.dropna(subset=['issue_class'])

In [9]:
agg_issue = issue.groupby('ad_id').agg({'issue_field': lambda x: '|'.join(x),
                                       'issue_class': lambda x: '|'.join(x)}).reset_index()

In [10]:
agg_issue.head(5)

,ad_id,issue_field,issue_class
0,x_1000009254731384,google_asr_text|aws_ocr_text_vid|ad_creative_body,"ISSUE215,ISSUE10,ISSUE83,ISSUE56|ISSUE215,ISSU..."
1,x_1000018871393117,ad_creative_link_description|page_name|ad_crea...,"ISSUE215,ISSUE12|ISSUE55|ISSUE55|ISSUE16"
2,x_1000032110674951,ad_creative_link_description|aws_ocr_text_img,ISSUE30|ISSUE30
3,x_1000047430680621,ad_creative_body,ISSUE30
4,x_1000072604727964,ad_creative_link_title|aws_ocr_text_img|ad_cre...,"ISSUE83|ISSUE83|ISSUE215,ISSUE83"


In [11]:
agg_issue.shape

(295098, 3)

In [ ]:
agg_issue.to_csv("../data/fb2022_output_processed_25.csv", index = False)